# QDC 2025 Challenge - Track B <br> QOBLIB - 最大独立集合問題（Maximum Independent Set Problem）

このチャレンジ用ノートブックでは、量子アルゴリズムを用いて複雑な組合せ最適化問題に取り組む方法を学びます。
具体的には 最大独立集合（Maximum Independent Set, MIS）問題を扱います。これは、互いに隣接（辺で接続）していない頂点からなる集合のうち、最大サイズのものを見つける問題です。この問題には、以下のような多くの実世界での応用があります。

* **ネットワークセキュリティ**：干渉しない通信チャネルの選択
* **スケジューリング**：衝突しないタスクやリソースの割り当て
* **分子生物学**：タンパク質相互作用ネットワークの解析
* **ソーシャルネットワーク**：互いに独立な影響力のあるノードの特定

このノートブックは、次の 4 つのパートに分かれています。
- Part I: 最大独立集合問題の背景を学び、 [Quantum Optimization Benchmarking Library (QOBLIB) リポジトリ](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library) から問題インスタンスを取得・読み込む方法を学びます。
- Part II: 新しく登場した[Qiskit Optimization Mapper Addon](https://github.com/Qiskit/qiskit-addon-opt-mapper)を用いて、この問題を QUBO（Quadratic Unconstrained Binary Optimization）形式に変換し、量子コンピュータで解ける形にします。
- Part III: [Qiskit Function](https://quantum.cloud.ibm.com/docs/en/guides/functions) の一例として、[Q-CTRL Optimization Solver](https://quantum.cloud.ibm.com/docs/en/guides/q-ctrl-optimization-solver) を使って問題を解く方法を紹介します。
- Part IV: いよいよチャレンジです！最大独立集合問題のより複雑なインスタンスに対して、Qiskit Function、独自に実装した QAOA、あるいは QDC を通じて学ぶ高度な手法など、手持ちの量子手法を自由に使って挑戦してください。

## Table of Contents

- [Part I: Understanding and Loading the Maximum Independent Set Problem (6 points)](#part-I)
- [Part II: QUBO Formulation (6 points)](#part-II)
- [Part III: Solving a MIS Problem with Q-CTRL Optimization Solver (MIS) (0 points)](#part-III)
- [Part IV: Main Challenge - Tackling a Complex Maximum Indepdent Set Problem Instance (88 points)](#part-IV)

# Imports

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.optimize import minimize
import requests
from urllib.parse import urlparse
import tempfile
import os

# Qiskit imports
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import qaoa_ansatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2 as Estimator, SamplerV2 as Sampler

# Q-CTRL Optimization Solver
from qiskit_ibm_catalog import QiskitFunctionsCatalog

# Import grading functions
from qc_grader.challenges.qdc_2025.qdc25_lab8 import (
    submit_name,
    grade_qctrl_function,
    grade_lab8_ex1,
    grade_lab8_ex2,
    grade_lab8_ex3,
)

from qiskit_addon_opt_mapper.applications import IndependentSet
from qiskit_addon_opt_mapper.converters import OptimizationProblemToQubo

# Sympy for polynomial representations
from sympy import symbols, Poly, srepr

print("All packages imported successfully!")

## Submit Team Name

In [ ]:
# Please use the exact spelling of your team name across teammates and across challenges

team_name = # Add team name string
submit_name(team_name)

<a id="part-I"></a>
# Part I：最大独立集合問題の理解と読み込み（6 点）

最大独立集合（Maximum Independent Set, MIS）問題とは、「与えられたグラフに対して、互いに辺で接続されていない頂点の集合のうち、最大のものは何か」を問う問題です。

## 問題の定式化

$n$ 個の頂点をもつグラフ $G = (V, E)$ に対して、任意の 2 つの頂点 $u, v \in S$ の間に辺 $(u,v) \in E$ が存在しないような最大の部分集合 $S \subseteq V$ を求めます。

これは次の最適化問題として定式化できます：
- **最大化**: $\sum_{i=1}^{n} x_i$ 
- **制約条件**：$x_i + x_j \leq 1$ for all edges $(i,j) \in E$
- ここで $x_i \in {0,1}$ は、頂点 $i$ が選ばれているかどうかを表します。

### 量子的定式化
QAOA を用いるために、ペナルティ項を加えて制約なし問題に変換します。
- **最小化**：$-\sum_{i=1}^{n} x_i + P \sum_{(i,j) \in E} x_i x_j$
- ここで $P$ は十分大きなペナルティ重みです。

この最適化問題は、次のハミルトニアンとして表現されます: $H = -\sum_{i=1}^{n} Z_i + P \sum_{(i,j) \in E} Z_i Z_j$

*注*：この演習は基本的な概念を示すことを目的としていますが、QPU クレジットを消費する可能性があります。メインのチャレンジ用にリソースを節約したい場合は、Exercise 3 にスキップしても構いません

## データの読み込みと問題フォーマット

<div class="alert alert-block alert-success">
    
<b>Exercise 1：`parse_gph_file` 関数を完成させてください（6 点）</b>

あなたの課題は、`.gph` ファイルの各行からデータを抽出するための パース処理（解析ロジック）を完成させることです。

**やるべきこと：**
- ファイルを 1 行ずつ読み込む
- コメント行をスキップする
- `p edge n m` という形式の行から、頂点数が `n`、辺数が `m` のグラフを作成する
- `e u v` で始まる行は、頂点 `u` と `v` の間に辺があることを表すので、それに基づいて辺を追加する

</div>

In [ ]:
# Parse GPH File Format
import networkx as nx
def parse_gph_file(filepath):
    """
    Parse a .gph file (DIMACS graph format) into a NetworkX graph.

    The .gph format specifications:
    - Lines starting with 'c' are comments
    - Line starting with 'p edge n m' defines a graph with n vertices and m edges
    - Lines starting with 'e u v' define an edge between vertices u and v

    Args:
        filepath (str): Path to the .gph file

    Returns:
        nx.Graph: Parsed graph
    """
    graph = nx.Graph()

    if not filepath or not os.path.exists(filepath):
        print("File not found. Please check the path or download manually.")
        return None

    with open(filepath, 'r') as f:
        lines = f.readlines()

    print(f"Parsing {len(lines)} lines...")

    for line_num, line in enumerate(lines, 1):
        line = line.strip()

        if not line or line.startswith('c'):
            # Skip comments and empty lines
            continue

        elif line.startswith('p edge'):
            # Parse problem definition: p edge <vertices> <edges>
            parts = line.split()
            if len(parts) >= 4:
                
                ### Don't change any code before this line ###
                

                
                ### Don't change any code past this line ###
        
        elif line.startswith('e'):
            # Parse edge: e <vertex1> <vertex2>
            parts = line.split()
            if len(parts) >= 3:
                try:
                    
                    ### Don't change any code before this line ###
                    

                    
                    ### Don't change any code before this line ###
                
                except ValueError:
                    print(f"Warning: Could not parse edge on line {line_num}: {line}")
        else:
            print(f"Warning: Unknown line format on line {line_num}: {line}")

    print(f"Successfully parsed graph:")
    print(f"   - Vertices: {len(graph.nodes)}")
    print(f"   - Edges: {len(graph.edges)}")
    print(f"   - Density: {nx.density(graph):.3f}")

    return graph

In [ ]:
# Submit your answer using following code

grade_lab8_ex1(parse_gph_file) # Expected result type: Callable

In [ ]:
# Download QOBLIB Instance

import requests
import os
import tempfile

qoblib_url = "https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/raw/main/07-independentset/instances/es60fst02.gph"

def download_qoblib_instance(url, filename=None):
    """
    Download a QOBLIB instance from the given URL.

    Args:
        url (str): URL to the .gph file
        filename (str, optional): Local filename to save to

    Returns:
        str: Path to the downloaded file
    """
    if filename is None:
        # Extract filename from URL
        filename = url.split('/')[-1].split('?')[0]

    print(f"Downloading {filename} from QOBLIB...")

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Save to the same directory as the notebook
        current_dir = os.getcwd()
        filepath = os.path.join(current_dir, filename)

        with open(filepath, 'wb') as f:
            f.write(response.content)

        print(f"Downloaded successfully to: {filepath}")
        print(f"File size: {len(response.content)} bytes")
        return filepath

    except requests.exceptions.RequestException as e:
        print(f"Download failed: {e}")
        print("Alternative: You can manually download from:")
        print(f"   {url}")
        print("   and place it in the same directory as this notebook")
        return None

downloaded_file = download_qoblib_instance(qoblib_url)

In [ ]:
# Parse the downloaded file
if downloaded_file:
    qoblib_graph = parse_gph_file(downloaded_file)
else:
    print("Attempting to load from current directory...")
    # Try to load from current directory if download failed
    local_file = "es60fst02.gph"
    if os.path.exists(local_file):
        qoblib_graph = parse_gph_file(local_file)
    else:
        print("Please download the file manually from the URL above")
        qoblib_graph = None

<a id="part-II"></a>
# Part II: QUBO 定式化 (6点)

In [ ]:
# Use Qiskit Addon Opt Mapper to generate the MIS Hamiltonian
def create_mis_graph(n_vertices=6, edge_probability=0.4, seed=42):
    """Create a random graph for the MIS problem."""
    np.random.seed(seed)
    G = nx.erdos_renyi_graph(n_vertices, edge_probability, seed=seed)
    return G

def visualize_mis_solution(graph, solution_bitstring):
    """Visualize the MIS solution on the graph."""
    pos = nx.spring_layout(graph, seed=42)
    node_colors = ['red' if solution_bitstring[i] == '1' else 'lightblue'
                   for i in range(len(graph.nodes))]

    plt.figure(figsize=(8, 6))
    nx.draw(graph, pos, node_color=node_colors, with_labels=True,
            node_size=500, font_size=16, font_weight='bold')
    plt.title("Maximum Independent Set Solution\n(Red = Selected vertices)")
    plt.show()

# Create a small example graph
mis_graph = create_mis_graph(n_vertices=6, edge_probability=0.3)
print(f"Created graph with {len(mis_graph.nodes)} vertices and {len(mis_graph.edges)} edges")

# Visualize the problem graph
pos = nx.spring_layout(mis_graph, seed=42)
plt.figure(figsize=(8, 6))
nx.draw(mis_graph, pos, with_labels=True, node_color='lightblue',
        node_size=500, font_size=16, font_weight='bold')
plt.title("MIS Problem Graph")
plt.show()

<a id="exercise2"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 2: QUBO への変換（6 点）</b>

[`qiskit_addon_opt_mapper`](https://github.com/Qiskit/qiskit-addon-opt-mapper)パッケージを用いて、
最大独立集合（Maximum Independent Set）問題を QUBO 形式に変換してください。

**課題:**
1. graphを入力として `IndepdentSet` オブジェクトを作成する
2. penalty parameter = 2として問題を QUBO 形式に変換する

**注**: このExerciseでは、 `create_mis_graph` 関数によって生成される小規模グラフ `mis_graph` を使用してください。最終Challengeでは、 `es60fst02` インスタンスを使用してください。
</div>

In [ ]:
# Convert to Hamiltonian using Qiskit Addon Opt Mapper
### Write your code below here ###

mis = 
qubo = 
mis_hamiltonian = 

### Don't change any code past this line ###
print(f"Hamiltonian generated with {len(mis_hamiltonian.paulis)} terms")
print("Sample terms:", mis_hamiltonian.paulis[:3])

In [ ]:
# Submit your answer using following code

grade_lab8_ex2(qubo) # Expected result type: OptimizationProblem

<a id="part-III"></a>
# Part III: Q-CTRL Optimization Solver を用いた MIS 問題の解法（0 点）

このパートでは、 [IBM's QAOA tutorial, Part II](https://quantum.cloud.ibm.com/docs/en/tutorials/quantum-approximate-optimization-algorithm#part-ii-scale-it-up)に従って 100 ノードのグラフに対する最大独立集合（MIS）問題を解きます。ただし、解法として Q-CTRL Optimization Solver を使用します。

実施する内容は以下のとおりです。

1. IBM チュートリアルで用いられている 100 ノードのグラフを構築する
2. Q-CTRL が推奨する手法を用いて、MIS 問題を sympy の多項式に変換する
3. 問題を Q-CTRL Optimization Solver に送信して解く

*注意*: この演習は基本的な概念の理解を目的としていますが、QPU クレジットを消費（約 20 分）します。メインのチャレンジ用にリソースを節約したい場合は、Exercise 3 にスキップしても構いません。

In [ ]:
# # If you haven't done so already
# # Uncomment the follow code to save the QDC 2025 account provided to you
#
# your_api_key = "deleteThisAndPasteYourAPIKeyHere"
# your_crn = "deleteThisAndPasteYourCRNHere"

# QiskitRuntimeService.save_account(
#     channel='ibm_quantum_platform',
#     token=your_api_key,
#     instance=your_crn,
#     name="qdc-2025"
# )

In [ ]:
# Load the Qiskit Functions Catalog
catalog = QiskitFunctionsCatalog(name="qdc-2025")

# You should see a list of Qiskit Functions available to you
# If you encounter the error `QiskitServerlessException: Credentials couldn't be verified`,
# it means your access is not yet active
display(catalog.list())

print("Successfully initialized Qiskit Functions Catalog")
print("Ready to load quantum optimization functions")

<a id="function-exercise"></a>
<div class="alert alert-block alert-success">
    
<b>Function Exercise: Q-CTRL Optimization Solver 関数の読み込み（0 点）</b>

catalogから Q-CTRL Optimization Solver の関数を読み込んでください。

**Tasks:**
1. Q-CTRL Optimization Solver に対応する 正しい関数名を特定する
2. `catalog.load()`を使って関数を読み込む
3. 読み込んだ結果を変数 `iskay_solver`に格納する

</div>

In [ ]:
### Write your code below here ###

function_name = ""  # TODO: Specify the Q-CTRL Optimization Solver
solver = catalog.load(function_name)

In [ ]:
# Submit your answer using following code

grade_qctrl_function(solver) # Expected result type: QiskitFunction

In [ ]:
# Q-CTRL Optimization Solver Demo for MIS (100-node example)
# QAOA Implementation for MIS (Optional - requires QPU credits)

# Uncomment and run this section if you want to execute on quantum hardware

"""
# Step 1: Create the 100-node MIS problem graph (from IBM QAOA tutorial Part II)
def create_100_node_mis_graph():
    # Create the 100-node graph from IBM's QAOA tutorial (Part II).
    # https://quantum.cloud.ibm.com/docs/en/tutorials/quantum-approximate-optimization-algorithm#part-ii-scale-it-up
    import itertools
    G = nx.Graph()
    G.add_nodes_from(range(100))
    # Edges: connect i to i+1 and i+2 (modulo 100)
    for i in range(100):
        G.add_edge(i, (i + 1) % 100)
        G.add_edge(i, (i + 2) % 100)
    return G

large_mis_graph = create_100_node_mis_graph()
print(f"Created graph with {len(large_mis_graph.nodes)} vertices and {len(large_mis_graph.edges)} edges")

# Visualize the problem graph (optional, can be slow for 100 nodes)
# pos = nx.spring_layout(large_mis_graph, seed=42)
# plt.figure(figsize=(10, 8))
# nx.draw(large_mis_graph, pos, with_labels=True, node_color='lightblue', node_size=200, font_size=8)
# plt.title("100-node MIS Problem Graph (IBM QAOA Tutorial Part II)")
# plt.show()

# Step 2: Convert the MIS problem to a sympy.Poly for Q-CTRL solver
from sympy import symbols, Poly, srepr

def mis_to_cost_polynomial(graph, penalty_weight=None):
    # Convert a Maximum Independent Set problem to a cost polynomial for Q-CTRL solver.
    # Minimize: -∑ᵢ xᵢ + P ∑_{(i,j) ∈ E} xᵢxⱼ
    vertices = list(graph.nodes())
    n_vertices = len(vertices)
    vertex_mapping = {v: i for i, v in enumerate(vertices)}
    variables = symbols([f'n[{i}]' for i in range(n_vertices)])
    if penalty_weight is None:
        penalty_weight = n_vertices + 1
    cost = 0
    for vertex in vertices:
        idx = vertex_mapping[vertex]
        cost -= variables[idx]
    for u, v in graph.edges():
        idx_u = vertex_mapping[u]
        idx_v = vertex_mapping[v]
        cost += penalty_weight * variables[idx_u] * variables[idx_v]
    polynomial = Poly(cost, *variables)
    return polynomial, vertex_mapping

large_mis_poly, large_vertex_mapping = mis_to_cost_polynomial(large_mis_graph)
print("MIS cost polynomial:")
print(large_mis_poly)

# Run Q-CTRL Optimization Solver (requires credentials and QPU credits)
cost_string = srepr(large_mis_poly)

print("Submitting 100-node MIS problem to Q-CTRL Optimization Solver...")
job = solver.run(
    problem=cost_string,
    backend_name=backend_name
)
print(f"Job submitted! Job ID: {job.job_id}")
print(f"Status: {job.status()}")

# To retrieve results after completion:
# result = job.result()
# solution_bitstring = result['solution_bitstring']
# final_cost = result['solution_bitstring_cost']
# print(f"Solution: {solution_bitstring}")
# print(f"Independent set size: {solution_bitstring.count('1')}")
"""

<a id="part-IV"></a>
# Part IV: Main Challenge ― 複雑な最大独立集合問題インスタンスへの挑戦（88 点）

いよいよメインチャレンジです！ここでは QOBLIB（Quantum Optimization Benchmark Library） に含まれる実際の最適化ベンチマーク問題を扱います。QOBLIB は、量子アルゴリズムの性能を評価するために用いられる、難易度の高い最適化問題の集合です。

<div class="alert alert-block alert-success">

<b>Exercise 3: `es60fst02` インスタンスを解く（88 点）</b>

`es60fst02` グラフは 186 個の頂点を持っており、QDC で利用可能な QPU の量子ビット数（156 量子ビット）を超えています。そのため、巧妙な近似戦略を考案する必要があります。

以下が取り組むべきステップです。
1. QOBLIB instanceを **ダウンロード** する 
2. グラフ形式を **パース(解析)** する
3. 問題サイズを戦略的に **縮小** する
4. Q-CTRL Optimizer 関数などの量子最適化アルゴリズムを用いて**解く**
5. 真の最適解にできるだけ近づくように**最適化**する
</div>

<div class="alert alert-block alert-warning">
  
このインスタンスに対する参照解（reference solution）は QOBLIB に用意されていますが、あなたの目標は、同程度の複雑さを持つ問題に対しても、同様の性能で解ける一般的かつスケーラブルな手法を設計することです。

> ⚠️ **重要:** 既知の答えを直接使用したり、それに依存した解法は失格となります。求められているのは、本物の量子的イノベーション―可能性の限界に挑む、創造的でアルゴリズム的なアプローチです！

提出期限後、上位入賞者のノートブックは専門家パネルによってレビューされ、追加の問題インスタンスを実行することで結果が検証されます。評価方法や提出手順の詳細については、[QDC contest rules](https://github.com/qiskit-community/qdc-challenges-2025/contest_rules.md) を参照してください。
</div>

<div class="alert alert-block alert-info">
以下に挙げるものを含め（ただしこれらに限定されず）、手持ちのあらゆる量子技術を自由に駆使して構いません。
    
- Qiskit functions
- 独自に実装した QAOA  
- 高度なトランスパイル手法や回路最適化戦略 
- 問題の別定式化
- 古典的な前処理・後処理のテクニック

アイデアの参考として、 [Quantum Optimization Best Practicies repository](https://github.com/qiskit-community/qopt-best-practices) を参照してください。
</div>

In [ ]:
# MIS Utility Functions (adapted from Q-CTRL documentation)

def mis_to_cost_polynomial(graph, vertex_mapping=None):
    """
    Convert a Maximum Independent Set problem to a cost polynomial for Q-CTRL solver.

    The MIS problem is formulated as:
    Minimize: -∑ᵢ xᵢ

    Args:
        graph (nx.Graph): The input graph
        vertex_mapping (dict): Optional mapping from graph vertices to variable indices

    Returns:
        sympy.Poly: Polynomial representation of the MIS cost function
    """
    from sympy import symbols, Poly

    vertices = list(graph.nodes())
    n_vertices = len(vertices)

    # Create vertex to index mapping if not provided
    if vertex_mapping is None:
        vertex_mapping = {v: i for i, v in enumerate(vertices)}

    # Create symbolic variables
    variables = symbols([f'n[{i}]' for i in range(n_vertices)])

    # Calculate penalty weight: should be larger than maximum possible reward
    penalty_weight = n_vertices + 1

    # Initialize cost function
    cost = 0

    # Reward terms: -xᵢ for each vertex (we want to maximize number of selected vertices)
    for vertex in vertices:
        idx = vertex_mapping[vertex]
        cost -= variables[idx]

    # Penalty terms: P * xᵢ * xⱼ for each edge (i,j)
    for u, v in graph.edges():
        idx_u = vertex_mapping[u]
        idx_v = vertex_mapping[v]
        cost += penalty_weight * variables[idx_u] * variables[idx_v]

    # Convert to sympy Poly
    polynomial = Poly(cost, *variables)

    return polynomial, vertex_mapping

def check_independent_set(graph, bitstring, vertex_mapping):
    """
    Check if a bitstring represents a valid independent set.

    Args:
        graph (nx.Graph): The graph
        bitstring (str): Binary string representing selected vertices
        vertex_mapping (dict): Mapping from vertices to bitstring indices

    Returns:
        bool: True if the set is independent, False otherwise
    """
    # Get selected vertices
    selected_vertices = []
    reverse_mapping = {v: k for k, v in vertex_mapping.items()}

    for i, bit in enumerate(bitstring):
        if bit == '1':
            selected_vertices.append(reverse_mapping[i])

    # Check if any two selected vertices are adjacent
    for u in selected_vertices:
        for v in selected_vertices:
            if u != v and graph.has_edge(u, v):
                return False

    return True

def maximum_independent_set_size(bitstring):
    """Calculate the size of the independent set represented by the bitstring."""
    return bitstring.count('1')

def analyze_graph_structure(graph):
    """Analyze key properties of the graph to inform reduction strategies."""
    print("Graph Analysis:")
    print(f"   Vertices: {len(graph.nodes)}")
    print(f"   Edges: {len(graph.edges)}")
    print(f"   Density: {nx.density(graph):.4f}")
    print(f"   Average degree: {sum(dict(graph.degree()).values()) / len(graph.nodes):.2f}")

    # Compute degree statistics
    degrees = [d for n, d in graph.degree()]
    print(f"   Degree range: {min(degrees)} - {max(degrees)}")

    # Find connected components
    components = list(nx.connected_components(graph))
    print(f"   Connected components: {len(components)}")
    if len(components) > 1:
        component_sizes = [len(c) for c in components]
        print(f"   Component sizes: {sorted(component_sizes, reverse=True)}")

    return {
        'vertices': len(graph.nodes),
        'edges': len(graph.edges),
        'density': nx.density(graph),
        'avg_degree': sum(degrees) / len(degrees),
        'max_degree': max(degrees),
        'min_degree': min(degrees),
        'components': len(components)
    }

# Analyze the QOBLIB graph if it was loaded successfully
if qoblib_graph is not None:
    graph_stats = analyze_graph_structure(qoblib_graph)

    # Visualize degree distribution
    degrees = [d for n, d in qoblib_graph.degree()]
    plt.figure(figsize=(10, 6))
    plt.hist(degrees, bins=20, edgecolor='black', alpha=0.7)
    plt.xlabel('Vertex Degree')
    plt.ylabel('Frequency')
    plt.title('Degree Distribution of es60fst02 Graph')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Graph not loaded - skipping analysis")

## The Challenge: グラフ縮約戦略

es60fst02 グラフは **186 個の頂点** を持っており、一般的な Heron デバイスにはまだ収まりません。そのため、最適解に関する情報をできるだけ保ったまま、問題サイズを削減する巧妙な戦略を考える必要があります。

### 戦略のアイデア:

1. **頂点の選択**:
   - 次数（接続数）が大きい頂点は、独立集合に含まれにくい
   - 次数が小さい頂点は、有力な候補となりやすい

2. **部分グラフの抽出**:
   - 複数の小さな部分問題として解く
   - 異なる部分グラフの解を組み合わせる

4. **ランダム化手法**:
   - 複数回の実行を伴うランダムサンプリング
   - モンテカルロ法によるアプローチ

<div class="alert alert-block alert-info">
<b>成功のためのヒント：</b>
    
**複数の戦略を試す**：さまざまなグラフ縮約手法をテストする  
**反復と改善**：初期の実行結果から得た知見を使って手法を改良する  
**プロセスを記録する**：戦略を明確に説明することで、検証がスムーズになる  
**結果を検証する**：得られた解が常に有効な独立集合であることを確認する

In [ ]:
# YOUR TURN: Implement your own strategy!
def your_custom_strategy(graph, target_size=150):
    """
    YOUR CUSTOM STRATEGY: Implement your own approach here!

    Return: (subgraph, mapping) where mapping maps original->new vertex labels
    """
    # TODO: Implement your strategy here
    # This is a placeholder - replace with your implementation

    # Example: Random sampling (you can do better!)
    vertices = list(graph.nodes())
    np.random.seed(42)
    selected_vertices = np.random.choice(vertices, min(target_size, len(vertices)), replace=False)

    subgraph = graph.subgraph(selected_vertices).copy()
    mapping = {old: new for new, old in enumerate(sorted(subgraph.nodes()))}
    subgraph = nx.relabel_nodes(subgraph, mapping)

    print(f"Your strategy: Selected {len(selected_vertices)} vertices")
    print(f"Subgraph has {len(subgraph.edges)} edges")

    return subgraph, mapping

# Test different strategies if graph is available
if qoblib_graph is not None:
    print("\nTesting reduction strategies:")
    print("=" * 50)

    # Test each strategy
    strategies = [
        ("Your custom strategy", your_custom_strategy)
    ]

    strategy_results = {}

    for name, strategy_func in strategies:
        print(f"\nTesting: {name}")
        try:
            reduced_graph, vertex_mapping = strategy_func(qoblib_graph, target_size=150)
            strategy_results[name] = (reduced_graph, vertex_mapping)
            print(f"   Success: {len(reduced_graph.nodes)} vertices, {len(reduced_graph.edges)} edges")
            print(f"   Density: {nx.density(reduced_graph):.4f}")
        except Exception as e:
            print(f"   Failed: {e}")
            strategy_results[name] = None
else:
    print("Original graph not available - implement strategies and test with your own graphs")

In [ ]:
# Solving the reduced graphs using quantum methods and find the solution of the original graph











solution_bitstring = 

<div class="alert alert-block alert-warning">
    
<b>スコア算出式</b>

スコア関数は次のように単純に定義されます：

$$\text{score} = \text{size of the indepdent set}$$

この問題インスタンスにおいて、最大独立集合のサイズは `88` であり、これが達成可能な 最高スコア です。

</div>

In [ ]:
# Submit your answer using following code

grade_lab8_ex3(solution_bitstring) # Expected result type: str

# References

1. [Q-CTRL Optimization Solver Documentation](https://docs.quantum.ibm.com/guides/q-ctrl-optimization-solver)
2. [IBM QAOA Tutorial](https://quantum.cloud.ibm.com/docs/en/tutorials/quantum-approximate-optimization-algorithm)
3. [QOBLIB Benchmark Library](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library)
4. [NetworkX Documentation](https://networkx.org/documentation/stable/)

# Additional Information

**Created by:** You Quan Chong, Yulun Wang

**Advised by:** Junye Huang

**Version:** 1.0.0

# Qiskit packages versions

In [ ]:
import qiskit
import qiskit_ibm_runtime
import qiskit_ibm_catalog

print(f'Qiskit: {qiskit.__version__}')
print(f'Qiskit IBM Runtime: {qiskit_ibm_runtime.__version__}')
print(f'Qiskit IBM Catalog: {qiskit_ibm_catalog.__version__}')